In [3]:
import requests
from bs4 import BeautifulSoup

def search_and_save_links(query, limit, ignored_sites=None, forbidden_words=None):
    # Задаем URL для поиска, например, Google
    url = f"https://www.google.com/search?q={query}"
    
    # Отправляем запрос
    response = requests.get(url)
    
    # Проверяем успешность запроса
    if response.status_code == 200:
        # Используем BeautifulSoup для парсинга HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Находим все ссылки на странице результатов поиска
        links = soup.find_all('a')
        
        # Счетчик для ограничения количества сохраняемых ссылок
        count = 0
        
        # Список для сохранения проверенных и подходящих ссылок
        saved_links = []
        
        # Проходимся по найденным ссылкам
        for link in links:
            href = link.get('href')
            
            # Проверяем, что ссылка не является ссылкой на результаты поиска, картой или изображением
            if href.startswith('/url?q=http') and not any(ext in href for ext in ['.google.', '/maps/', '/search?q=related:', '/imgres?']):
                # Извлекаем саму ссылку
                clean_link = href.split('/url?q=')[1].split('&')[0]
                
                # Проверяем, что ссылка не на Википедию на любом языке
                if "wikipedia.org" in clean_link:
                    continue
                
                # Проверяем, что ссылка не находится в списке игнорируемых сайтов
                if ignored_sites and any(site in clean_link for site in ignored_sites):
                    continue
                
                # Проверяем содержимое веб-страницы на наличие запрещенных слов
                if forbidden_words:
                    page_response = requests.get(clean_link)
                    if page_response.status_code == 200:
                        page_content = page_response.text.lower()
                        if any(word in page_content for word in forbidden_words):
                            continue
                
                # Добавляем ссылку в список сохраненных ссылок
                saved_links.append(clean_link)
                
                # Увеличиваем счетчик сохраненных ссылок
                count += 1
                
                # Если количество сохраненных ссылок достигло лимита, прерываем цикл
                if count == limit:
                    break
        
        # Возвращаем список сохраненных ссылок
        return saved_links
    else:
        # Если запрос не удался, выводим сообщение об ошибке
        print("Ошибка при отправке запроса")
        return None

# Пример использования функции
query = input("Введите ваш запрос: ")
limit = int(input("Введите количество ссылок для сохранения: "))

ignored_sites_input = "https://wikipedia.org, https://www.youtube.com,  https://www.russianfood.com"
ignored_sites = [site.strip() for site in ignored_sites_input.split(",")] if ignored_sites_input else None

forbidden_words_input = ""
forbidden_words = [word.strip().lower() for word in forbidden_words_input.split(",")] if forbidden_words_input else None

links = search_and_save_links(query, limit, ignored_sites, forbidden_words)
if links:
    print("Сохраненные ссылки:")
    for idx, link in enumerate(links, start=1):
        print(f"{idx}. {link}")


Введите ваш запрос: концертный зал холл теракт сити крокус
Введите количество ссылок для сохранения: 10
Сохраненные ссылки:
1. https://tass.ru/proisshestviya/20320557
2. https://29.ru/text/incidents/2024/03/25/73379672/
3. https://78.ru/articles/2024-03-23/krokus-siti-holl
4. https://news.ru/society/pochemu-terroristy-napali-na-krokus-siti-holl-chto-eto-za-koncertnyj-zal/
5. https://www.dw.com/ru/napadenie-na-koncertnyj-zal-krokus-siti-holl-v-podmoskove/live-68649000
6. https://iz.ru/1670070/izvestiia/strelba-na-kontcerte-v-moskve-neizvestnye-otkryli-ogon-v-krokus-siti-kholle
7. https://www.interfax-russia.ru/center/main/koncertnyy-zal-crocus-city-hall-gorit-iz-za-vooruzhennogo-napadeniya-est-pogibshie
8. https://www.aa.com.tr/ru/%25D0%25BC%25D0%25B8%25D1%2580/%25D1%2584%25D1%2581%25D0%25B1-%25D1%2580%25D0%25BE%25D1%2581%25D1%2581%25D0%25B8%25D0%25B8-%25D0%25B2-%25D1%2580%25D0%25B5%25D0%25B7%25D1%2583%25D0%25BB%25D1%258C%25D1%2582%25D0%25B0%25D1%2582%25D0%25B5-%25D1%2582%25D0%25B5%25D1

In [16]:
import requests
from bs4 import BeautifulSoup
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def extract_useful_text(url):
    try:
        # Отправляем запрос на указанный URL
        response = requests.get(url)
        
        # Проверяем успешность запроса
        if response.status_code == 200:
            # Используем BeautifulSoup для парсинга HTML
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Извлекаем текстовую информацию из HTML-кода
            text = soup.get_text()
            
            # Удаляем лишние символы (пунктуация и т.д.)
            text = re.sub(r'[^\w\s]', '', text)
            
            # Токенизируем текст
            tokens = word_tokenize(text)
            
            # Удаляем стоп-слова
            stop_words = set(stopwords.words('english'))  # Указываем язык текста
            filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
            
            # Лемматизируем слова
            lemmatizer = WordNetLemmatizer()
            lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]
            
            # Склеиваем токены обратно в текст
            useful_text = ' '.join(lemmatized_tokens)
            
            return useful_text
        else:
            print(f"Ошибка при получении содержимого сайта {url}: {response.status_code}")
            return None
    except Exception as e:
        print(f"Ошибка: {e}")
        return None

# Пример использования функции
urls = [
"https://www.britannica.com/event/Columbine-High-School-shootings",
"https://www.cbsnews.com/news/columbine-25-years-later-school-lockdown-active-shooter-drills/",
"https://www.usatoday.com/story/news/nation/2024/04/19/columbine-mass-school-shooting-anniversary-trauma/73203075007/",
"https://www.history.com/topics/1990s/columbine-high-school-shootings",
"https://www.washingtonpost.com/education/interactive/school-shootings-database/"
]

for url in urls:
    print(f"Полезная информация с сайта {url}:")
    text = extract_useful_text(url)
    if text:
        print(text)
        print("-" * 50)


Полезная информация с сайта https://www.britannica.com/event/Columbine-High-School-shootings:
Columbine High School shooting Date Shooters History Facts Britannica Search Britannica Click search Search Britannica Click search Login Subscribe Subscribe Home Games Quizzes History Society Science Tech Biographies Animals Nature Geography Travel Arts Culture Money Videos Day One Good Fact Dictionary New Articles History Society Lifestyles Social Issues Philosophy Religion Politics Law Government World History Science Tech Health Medicine Science Technology Biographies Browse Biographies Animals Nature Birds Reptiles Vertebrates Bugs Mollusks Invertebrates Environment Fossils Geologic Time Mammals Plants Geography Travel Geography Travel Arts Culture Entertainment Pop Culture Literature Sports Recreation Visual Arts Companions Demystified Image Galleries Infographics Lists Podcasts Spotlights Summaries Forum Top Questions WTFact 100 Women Britannica Kids Saving Earth Space Next 50 Student C

Columbine High School shooting still impact u 25 year laterInside courtroom College protest Start day smarter Bird color explainedUS Elections Sports Entertainment Life Money Tech Travel OpinionONLY USA TODAYNewsletters Subscribers Archives Crossword eNewspaper Magazines Investigations Podcasts Video Humankind Curious Bestselling BooklistOUR PORTFOLIO10Best Reviewed Coupons Homefront Blueprint Best Auto InsuranceBest Pet Insurance Best Travel Insurance Best Credit Cards Best CD Rates Best Personal Loans Home Internet NATIONColumbine High School MassacreAdd Topic25 year later trauma Columbine High School shooting still u Ndea YanceyBraggUSA TODAYDave Cullen sat lunch Budget Gourmet frozen meal beef stroganoff medium first reported shot fired school Littleton Colorado warm April day quarter century agoJaclyn Schildkraut home sick freshman year college watching soap opera Days Lives think news broke aerial video SWAT team terrified student running Columbine High School hand headsRobert Th

Columbine High School Shooting Victims Killers HISTORYSkip contentShowsThis Day HistoryScheduleTopicsStoriesHistory ClassicsLive TVYour ProfileYour ProfileHistoryFind History Facebook Opens new windowFind History Twitter Opens new windowFind History YouTube Opens new windowFind History Instagram Opens new windowFind History TikTok Opens new windowEmail UpdatesLive TVHistory ClassicsShowsThis Day HistoryScheduleTopicsStoriesVideosHistory PodcastsHistory VaultHomeTopics1990sColumbine ShootingColumbine ShootingBy Historycom EditorsUpdated June 29 2023 Original November 9 2009copy page linkPrint PageCHARLES SCHAEFERAFP via Getty ImagesTable ContentsDylan Klebold Eric Harris Columbine Shooting VictimsShe Said YesColumbine Shooting Investigation Columbine Massacre Aftermath Columbine shooting April 20 1999 Columbine High School Littleton Colorado occurred two teen went shooting spree killing 13 people wounding 20 others turning gun committing suicide Columbine shooting time worst high school